In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
from __future__ import division, print_function

import collections
import csv
import datetime
import xml.etree.ElementTree as ET

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from scipy.interpolate import CubicSpline

from GlucNet_functions import *

In [3]:
filepath = f"../OhioT1DM/2018/train/559-ws-training.xml"
tree = ET.parse(filepath)
root = tree.getroot()
glucose = read_ohio(filepath, "glucose_level", False)

# Preprocessing

#### P1

In [4]:
glucose_dict = {entry[0]['ts']: entry[0]['value'] for entry in glucose}

# Create the multi-channel database
g_data = []
for timestamp in glucose_dict:
    record = {
        'timestamp': timestamp,
        'glucose_value': glucose_dict[timestamp],
        # 'meal_type': None,
        # 'meal_carbs': 0
    }
    
    g_data.append(record)
glucose_df = pd.DataFrame(g_data)

# Convert glucose values to numeric type for analysis
glucose_df['glucose_value'] = pd.to_numeric(glucose_df['glucose_value'])

# Calculate percentiles
lower_percentile = np.percentile(glucose_df['glucose_value'], 2)
upper_percentile = np.percentile(glucose_df['glucose_value'], 98)

# Print thresholds
print(f"2% lower threshold: {lower_percentile}")
print(f"98% upper threshold: {upper_percentile}")

2% lower threshold: 64.0
98% upper threshold: 337.0


### P2, P3

In [5]:
segments = segement_data_as_1hour(glucose_df)
interpolated_segements = detect_missing_and_spline_interpolate(segments)

segment_3
before:  2021-12-12 04:54:00
after:  2021-12-12 05:29:00
segment_3
before:  2021-12-12 12:34:00
after:  2021-12-12 12:44:00
segment_3
before:  2021-12-12 22:09:00
after:  2021-12-12 22:39:00
segment_7
before:  2021-12-16 17:53:00
after:  2021-12-16 18:18:00
segment_14
before:  2021-12-22 17:52:00
after:  2021-12-22 18:52:00
segment_14
before:  2021-12-24 17:52:00
after:  2021-12-24 18:32:00
segment_15
before:  2021-12-26 18:57:00
after:  2021-12-26 19:22:00
segment_18
before:  2022-01-01 20:50:00
after:  2022-01-01 21:10:00
segment_21
before:  2022-01-06 06:26:00
after:  2022-01-06 07:21:00
segment_25
before:  2022-01-08 18:05:00
after:  2022-01-08 18:40:00
segment_29
before:  2022-01-14 01:05:00
after:  2022-01-14 01:35:00
segment_31
before:  2022-01-16 19:11:00
after:  2022-01-16 19:56:00


/dartfs-hpc/rc/home/j/f007g3j/ReproGenBG_ML4H/GluNet_Again/GlucNet_functions.py:35: FutureWarning: 'T' is deprecated and will be removed in a future version, please use 'min' instead.
  datetime_list = np.array(pd.date_range(start=min(segments[sequence]['timestamp']), end=max(segments[sequence]['timestamp']), freq='5T').tolist())
/dartfs-hpc/rc/home/j/f007g3j/ReproGenBG_ML4H/GluNet_Again/GlucNet_functions.py:48: FutureWarning: 'T' is deprecated and will be removed in a future version, please use 'min' instead.
  time_index_interpolated = pd.date_range(start=reference_time, periods=len(interpolated_xs), freq='5T')


In [6]:
meal = read_ohio(filepath, "meal", False)

flattened_meal_data = [item[0] for item in meal]  # Take the first (and only) item from each sublist

# Convert to DataFrame
meal_df = pd.DataFrame(flattened_meal_data)

meal_df['assigned'] = False

meal_updated_segments = update_segments_with_meals(interpolated_segements, meal_df)


/dartfs-hpc/rc/home/j/f007g3j/ReproGenBG_ML4H/GluNet_Again/GlucNet_functions.py:421: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '18' has dtype incompatible with int64, please explicitly cast to a compatible dtype first.
  segment_df.at[i, 'carbs'] = closest_meal['carbs']
/dartfs-hpc/rc/home/j/f007g3j/ReproGenBG_ML4H/GluNet_Again/GlucNet_functions.py:421: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '45' has dtype incompatible with int64, please explicitly cast to a compatible dtype first.
  segment_df.at[i, 'carbs'] = closest_meal['carbs']
/dartfs-hpc/rc/home/j/f007g3j/ReproGenBG_ML4H/GluNet_Again/GlucNet_functions.py:421: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '63' has dtype incompatible with int64, please explicitly cast to a compa

In [7]:
basal = read_ohio(filepath, "basal", False)

flattened_basal_data = [item[0] for item in basal]  # Take the first (and only) item from each sublist

# Convert to DataFrame
basal_df = pd.DataFrame(flattened_basal_data)

basal_df['assigned'] = False
basal_df['end_ts'] = basal_df['ts'].shift(-1)
basal_df[:10]

,ts,value,assigned,end_ts
0,2021-12-07 00:00:00,0.65,False,2021-12-07 04:00:00
1,2021-12-07 04:00:00,0.73,False,2021-12-07 08:00:00
2,2021-12-07 08:00:00,1.15,False,2021-12-07 11:00:00
3,2021-12-07 11:00:00,0.9,False,2021-12-08 00:00:00
4,2021-12-08 00:00:00,0.65,False,2021-12-08 04:00:00
5,2021-12-08 04:00:00,0.73,False,2021-12-08 08:00:00
6,2021-12-08 08:00:00,1.15,False,2021-12-08 11:00:00
7,2021-12-08 11:00:00,0.9,False,2021-12-08 18:00:00
8,2021-12-08 18:00:00,1.25,False,2021-12-11 00:00:00
9,2021-12-11 00:00:00,0.65,False,2021-12-11 04:00:00


In [8]:
basal_updated_segments = update_segments_with_basal(meal_updated_segments, basal_df)


In [9]:
# Merge Bolus into the dataframe
bolus = read_ohio_bolus_tempbasal(filepath, "bolus", False)

flattened_bolus_data = [item[0] for item in bolus]  # Take the first (and only) item from each sublist

# Convert to DataFrame
bolus_df = pd.DataFrame(flattened_bolus_data)

bolus_df['assigned'] = False
bolus_df[:10]

,ts_begin,ts_end,type,dose,bwz_carb_input,assigned
0,2021-12-07 01:08:04,2021-12-07 01:08:04,normal,1.6,25,False
1,2021-12-07 11:21:49,2021-12-07 11:21:49,normal,9.3,0,False
2,2021-12-07 23:50:34,2021-12-07 23:50:34,normal,3.8,0,False
3,2021-12-08 05:11:38,2021-12-08 05:11:38,normal,4.1,45,False
4,2021-12-08 11:27:23,2021-12-08 11:27:23,normal,6.3,63,False
5,2021-12-08 19:06:34,2021-12-08 19:06:34,normal,2.2,20,False
6,2021-12-09 02:05:46,2021-12-09 02:05:46,normal,0.6,0,False
7,2021-12-09 08:20:50,2021-12-09 08:20:50,normal,4.2,0,False
8,2021-12-09 13:15:16,2021-12-09 13:15:16,normal,2.0,20,False
9,2021-12-09 15:03:10,2021-12-09 15:03:10,normal,4.5,45,False


In [10]:
bolus_updated_segments = update_segments_with_bolus(basal_updated_segments, bolus_df)

/dartfs-hpc/rc/home/j/f007g3j/ReproGenBG_ML4H/GluNet_Again/GlucNet_functions.py:312: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '9.3' has dtype incompatible with int64, please explicitly cast to a compatible dtype first.
  segment_df.at[i, 'bolus_dose'] = closest_bolus['dose']
/dartfs-hpc/rc/home/j/f007g3j/ReproGenBG_ML4H/GluNet_Again/GlucNet_functions.py:312: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '4.1' has dtype incompatible with int64, please explicitly cast to a compatible dtype first.
  segment_df.at[i, 'bolus_dose'] = closest_bolus['dose']
/dartfs-hpc/rc/home/j/f007g3j/ReproGenBG_ML4H/GluNet_Again/GlucNet_functions.py:312: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '5.5' has dtype incompatible with int64, please explicitly ca

In [11]:
tempbasal = read_ohio_bolus_tempbasal(filepath, "temp_basal", False)

In [12]:
flattened_tempbasal_data = [item[0] for item in tempbasal]  # Take the first (and only) item from each sublist

# Convert to DataFrame
tempbasal_df = pd.DataFrame(flattened_tempbasal_data)

tempbasal_df['assigned'] = False
tempbasal_df[:10]

,ts_begin,ts_end,value,assigned
0,2021-12-07 04:49:28,2021-12-07 05:08:39,0.0,False
1,2021-12-07 17:01:44,2021-12-07 18:03:20,0.0,False
2,2021-12-08 15:12:19,2021-12-08 16:12:27,0.0,False
3,2021-12-09 11:11:10,2021-12-09 12:19:17,0.0,False
4,2021-12-10 14:14:00,2021-12-10 16:14:00,0.0,False
5,2021-12-10 19:51:28,2021-12-10 21:30:37,0.0,False
6,2021-12-11 17:25:48,2021-12-11 18:10:10,0.0,False
7,2021-12-15 01:32:00,2021-12-15 03:30:00,0.0,False
8,2021-12-17 20:24:00,2021-12-17 22:24:00,0.0,False
9,2021-12-22 19:52:33,2021-12-22 21:52:00,0.0,False


In [13]:


# Update the segments with meal data
final_updated_segments = update_segments_with_tempbasal(bolus_updated_segments, tempbasal_df)

# Training

In [14]:
import torch
from torch.utils.data import DataLoader, TensorDataset

In [23]:
def label_delta_transform(labels_list):
    # label_lower_percentile = -12.75
    # label_upper_percentile = 12.85
    label_lower_percentile = np.percentile(labels_list, 10)
    label_upper_percentile = np.percentile(labels_list, 90)
    transformed_labels = []
    for label in labels_list:
        if label <= label_lower_percentile:
            transformed_labels.append(1)
        elif label_lower_percentile < label < label_upper_percentile:
            trans_label = round((256/(label_upper_percentile - label_lower_percentile))*(label + abs(label_lower_percentile) + 0.05))
            transformed_labels.append(trans_label)
        elif label >= label_upper_percentile:
            transformed_labels.append(256)
    return transformed_labels


def prepare_dataset(segments, ph):
    '''
    ph = 6, 30 minutes ahead
    ph = 12, 60 minutes ahead
    '''
    features_list = []
    labels_list = []
    raw_glu_list = []
    
    history_len = 15
    
    # Iterate over each segment
    for segment_name, segment_df in segments.items():
        # Ensure all columns are of numeric type
        segment_df['carbs'] = pd.to_numeric(segment_df['carbs'], errors='coerce')
        segment_df['basal_rate'] = pd.to_numeric(segment_df['basal_rate'], errors='coerce')
        segment_df['bolus_dose'] = pd.to_numeric(segment_df['bolus_dose'], errors='coerce')

        # Fill NaNs that might have been introduced by conversion errors
        segment_df.fillna(0, inplace=True)

        # Maximum index for creating a complete feature set
        print("len of segment_df is ", len(segment_df))
        max_index = len(segment_df) - (history_len + ph)  # Subtracting only 15+ph to ensure i + 15 + ph is within bounds
        
        # Iterate through the data to create feature-label pairs
        for i in range(max_index):
            # Extracting features from index i to i+15
            segment_df = segment_df.reset_index(drop = True)
            features = segment_df.loc[i:i+history_len, ['glucose_value', 'carbs', 'basal_rate', 'bolus_dose']].values
            # Extracting label for index i+15+ph
            # label = segment_df.loc[i+15+ph, 'glucose_value'] - segment_df.loc[i+15, 'glucose_value']
            
            raw_glu_list.append(segment_df.loc[i+history_len+ph, 'glucose_value'])
            features_list.append(features)
            # labels_list.append(label)
            
    print("len of features_list " + str(len(features_list)))
    # print("len of labels_list " + str(len(labels_list)))
    
    # new_labels_list = label_delta_transform(labels_list)    
    # print("after label transform, the len of label list "+str(len(new_labels_list)))    
    
    return features_list, raw_glu_list


In [25]:
features_list, labels_list = prepare_dataset(segments, 6)

len of segment_df is  228
len of segment_df is  958
len of segment_df is  635
len of segment_df is  26
len of segment_df is  70
len of segment_df is  412
len of segment_df is  287
len of segment_df is  142
len of segment_df is  346
len of segment_df is  142
len of segment_df is  258
len of segment_df is  70
len of segment_df is  142
len of segment_df is  1161
len of segment_df is  272
len of segment_df is  70
len of segment_df is  1291
len of segment_df is  299
len of segment_df is  433
len of segment_df is  174
len of segment_df is  504
len of segment_df is  153
len of segment_df is  272
len of segment_df is  21
len of segment_df is  212
len of segment_df is  228
len of segment_df is  251
len of segment_df is  429
len of segment_df is  491
len of segment_df is  364
len of segment_df is  525
len of features_list 10215


In [26]:
features_array = np.array(features_list)
labels_array = np.array(labels_list)

# Step 1: Split into 80% train+val and 20% test
X_temp, X_test, y_temp, y_test = train_test_split(features_array, labels_array, test_size=0.2, shuffle=False)

# Step 2: Split the 80% into 70% train and 10% val (0.7/0.8 = 0.875)
X_train, X_val, y_train, y_val = train_test_split(X_temp, y_temp, test_size=0.125, shuffle=False)

# Convert the splits to torch tensors
X_train = torch.tensor(X_train, dtype=torch.float32)
y_train = torch.tensor(y_train, dtype=torch.float32)
X_val = torch.tensor(X_val, dtype=torch.float32)
y_val = torch.tensor(y_val, dtype=torch.float32)
X_test = torch.tensor(X_test, dtype=torch.float32)
y_test = torch.tensor(y_test, dtype=torch.float32)

# Create DataLoaders
train_dataset = TensorDataset(X_train, y_train)
train_loader = DataLoader(train_dataset, batch_size=32, shuffle=False)

val_dataset = TensorDataset(X_val, y_val)
val_loader = DataLoader(val_dataset, batch_size=32, shuffle=False)

test_dataset = TensorDataset(X_test, y_test)
test_loader = DataLoader(test_dataset, batch_size=32, shuffle=False)

In [27]:
# Convert lists to PyTorch tensors
features_tensor = torch.tensor(features_list, dtype=torch.float32)
labels_tensor = torch.tensor(labels_list, dtype=torch.float32).unsqueeze(1)  # Making labels tensor 2D

feature_label_tensor = TensorDataset(features_tensor, labels_tensor)


train_loader = DataLoader(feature_label_tensor, batch_size=32, shuffle=True)

# Example of using DataLoader in a training loop
for features, labels in train_loader:
    print("Features batch shape:", features.shape)
    print("Label batch shape:", labels.shape)
    # Example: print(features, labels)
    break

Features batch shape: torch.Size([32, 16, 4])
Label batch shape: torch.Size([32, 1])


In [28]:
# initialize cuda option
dtype = torch.FloatTensor # data type
ltype = torch.LongTensor # label type

use_cuda = torch.cuda.is_available()
if use_cuda:
    print('use gpu')
    dtype = torch.cuda.FloatTensor
    ltype = torch.cuda.LongTensor

In [29]:
# Build the dilate CNN
import torch
import torch.nn as nn
import torch.nn.functional as F

class WaveNetBlock(nn.Module):
    def __init__(self, in_channels, dilation):
        super(WaveNetBlock, self).__init__()
        self.conv1 = nn.Conv1d(in_channels, in_channels, kernel_size=2, dilation=dilation, padding=1+dilation - 2^(dilation-1))
        self.conv2 = nn.Conv1d(in_channels, in_channels, kernel_size=2, dilation=dilation, padding=dilation)
        self.res_conv = nn.Conv1d(in_channels, in_channels, kernel_size=1)
        
    def forward(self, x):
        # print("shape of x: ", x.shape)
        out = F.relu(self.conv1(x))
        # print("shape of first out: ", out.shape)
        out = F.relu(self.conv2(out))
        # print("shape of second out: ", out.shape)
        res = self.res_conv(x)
        # print("shape of res: ", res.shape)
        return out + res

class WaveNet(nn.Module):
    def __init__(self, in_channels, out_channels, num_blocks, dilations):
        super(WaveNet, self).__init__()
        self.initial_conv = nn.Conv1d(in_channels, 32, kernel_size=2, padding=1)
        self.blocks = nn.ModuleList([WaveNetBlock(32, dilation) for dilation in dilations])
        self.final_conv1 = nn.Conv1d(32, 128, kernel_size=2, padding=0)
        self.final_conv2 = nn.Conv1d(128, 256, kernel_size=2, padding=0)
        self.fc1 = nn.Linear(256, 128)
        self.fc2 = nn.Linear(128, out_channels)
        
    def forward(self, x):
        x = F.relu(self.initial_conv(x))
        for block in self.blocks:
            # print("enter the block loop")
            x = block(x)
        x = F.relu(self.final_conv1(x))
        x = F.relu(self.final_conv2(x))
        x = x[:, :, -1]  # Get the last time step
        x = F.relu(self.fc1(x))
        x = self.fc2(x)
        return x

input_channels = 4  # Number of features
output_channels = 1  # Predicting a single value (glucose level)
num_blocks = 4  # Number of WaveNet blocks
dilations = [2**i for i in range(num_blocks)]  # Dilation rates: 1, 2, 4, 8

model = WaveNet(input_channels, output_channels, num_blocks, dilations)
print(model)

# Example of how to define the loss and optimizer
criterion = nn.MSELoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.0008)


WaveNet(
  (initial_conv): Conv1d(4, 32, kernel_size=(2,), stride=(1,), padding=(1,))
  (blocks): ModuleList(
    (0): WaveNetBlock(
      (conv1): Conv1d(32, 32, kernel_size=(2,), stride=(1,))
      (conv2): Conv1d(32, 32, kernel_size=(2,), stride=(1,), padding=(1,))
      (res_conv): Conv1d(32, 32, kernel_size=(1,), stride=(1,))
    )
    (1): WaveNetBlock(
      (conv1): Conv1d(32, 32, kernel_size=(2,), stride=(1,), dilation=(2,))
      (conv2): Conv1d(32, 32, kernel_size=(2,), stride=(1,), padding=(2,), dilation=(2,))
      (res_conv): Conv1d(32, 32, kernel_size=(1,), stride=(1,))
    )
    (2): WaveNetBlock(
      (conv1): Conv1d(32, 32, kernel_size=(2,), stride=(1,), dilation=(4,))
      (conv2): Conv1d(32, 32, kernel_size=(2,), stride=(1,), padding=(4,), dilation=(4,))
      (res_conv): Conv1d(32, 32, kernel_size=(1,), stride=(1,))
    )
    (3): WaveNetBlock(
      (conv1): Conv1d(32, 32, kernel_size=(2,), stride=(1,), dilation=(8,))
      (conv2): Conv1d(32, 32, kernel_size=(2

In [30]:
for inputs, targets in train_loader:
    print("Input tensor shape:", inputs.shape)
    print("Input tensor total elements:", inputs.numel())
    print("Target tensor shape:", targets.shape)
    print("Sequence length:", inputs.shape[1])
    break


Input tensor shape: torch.Size([32, 16, 4])
Input tensor total elements: 2048
Target tensor shape: torch.Size([32, 1])
Sequence length: 16


In [32]:
features_list, labels_list = prepare_dataset(final_updated_segments, 6)

len of segment_df is  228
len of segment_df is  958
len of segment_df is  635
len of segment_df is  26
len of segment_df is  70
len of segment_df is  412
len of segment_df is  287
len of segment_df is  142
len of segment_df is  346
len of segment_df is  142
len of segment_df is  258
len of segment_df is  70
len of segment_df is  142
len of segment_df is  1161
len of segment_df is  272
len of segment_df is  70
len of segment_df is  1291
len of segment_df is  299
len of segment_df is  433
len of segment_df is  174
len of segment_df is  504
len of segment_df is  153
len of segment_df is  272
len of segment_df is  21
len of segment_df is  212
len of segment_df is  228
len of segment_df is  251
len of segment_df is  429
len of segment_df is  491
len of segment_df is  364
len of segment_df is  525
len of features_list 10215


In [33]:
features_array = np.array(features_list)
labels_array = np.array(labels_list)

# Step 1: Split into 80% train+val and 20% test
X_temp, X_test, y_temp, y_test = train_test_split(features_array, labels_array, test_size=0.2, shuffle=False)

# Step 2: Split the 80% into 70% train and 10% val (0.7/0.8 = 0.875)
X_train, X_val, y_train, y_val = train_test_split(X_temp, y_temp, test_size=0.125, shuffle=False)

# Convert the splits to torch tensors
X_train = torch.tensor(X_train, dtype=torch.float32)
y_train = torch.tensor(y_train, dtype=torch.float32)
X_val = torch.tensor(X_val, dtype=torch.float32)
y_val = torch.tensor(y_val, dtype=torch.float32)
X_test = torch.tensor(X_test, dtype=torch.float32)
y_test = torch.tensor(y_test, dtype=torch.float32)

# Create DataLoaders
train_dataset = TensorDataset(X_train, y_train)
train_loader = DataLoader(train_dataset, batch_size=32, shuffle=False)

val_dataset = TensorDataset(X_val, y_val)
val_loader = DataLoader(val_dataset, batch_size=32, shuffle=False)

test_dataset = TensorDataset(X_test, y_test)
test_loader = DataLoader(test_dataset, batch_size=32, shuffle=False)

In [34]:
features_array = np.array(features_list)
labels_array = np.array(labels_list)

# Step 1: Split into 80% train+val and 20% test
X_temp, X_test, y_temp, y_test = train_test_split(features_array, labels_array, test_size=0.2, shuffle=False)

# Step 2: Split the 80% into 70% train and 10% val (0.7/0.8 = 0.875)
X_train, X_val, y_train, y_val = train_test_split(X_temp, y_temp, test_size=0.125, shuffle=False)

# Convert the splits to torch tensors
X_train = torch.tensor(X_train, dtype=torch.float32)
y_train = torch.tensor(y_train, dtype=torch.float32)
X_val = torch.tensor(X_val, dtype=torch.float32)
y_val = torch.tensor(y_val, dtype=torch.float32)
X_test = torch.tensor(X_test, dtype=torch.float32)
y_test = torch.tensor(y_test, dtype=torch.float32)

# Create DataLoaders
train_dataset = TensorDataset(X_train, y_train)
train_loader = DataLoader(train_dataset, batch_size=32, shuffle=False)

val_dataset = TensorDataset(X_val, y_val)
val_loader = DataLoader(val_dataset, batch_size=32, shuffle=False)

test_dataset = TensorDataset(X_test, y_test)
test_loader = DataLoader(test_dataset, batch_size=32, shuffle=False)

In [35]:
# Convert lists to PyTorch tensors
features_tensor = torch.tensor(features_list, dtype=torch.float32)
labels_tensor = torch.tensor(labels_list, dtype=torch.float32).unsqueeze(1)  # Making labels tensor 2D

feature_label_tensor = TensorDataset(features_tensor, labels_tensor)


train_loader = DataLoader(feature_label_tensor, batch_size=32, shuffle=True)

# Example of using DataLoader in a training loop
for features, labels in train_loader:
    print("Features batch shape:", features.shape)
    print("Label batch shape:", labels.shape)
    # Example: print(features, labels)
    break

Features batch shape: torch.Size([32, 16, 4])
Label batch shape: torch.Size([32, 1])


In [36]:
# initialize cuda option
dtype = torch.FloatTensor # data type
ltype = torch.LongTensor # label type

use_cuda = torch.cuda.is_available()
if use_cuda:
    print('use gpu')
    dtype = torch.cuda.FloatTensor
    ltype = torch.cuda.LongTensor

In [37]:
# Build the dilate CNN
import torch
import torch.nn as nn
import torch.nn.functional as F

class WaveNetBlock(nn.Module):
    def __init__(self, in_channels, dilation):
        super(WaveNetBlock, self).__init__()
        self.conv1 = nn.Conv1d(in_channels, in_channels, kernel_size=2, dilation=dilation, padding=1+dilation - 2^(dilation-1))
        self.conv2 = nn.Conv1d(in_channels, in_channels, kernel_size=2, dilation=dilation, padding=dilation)
        self.res_conv = nn.Conv1d(in_channels, in_channels, kernel_size=1)
        
    def forward(self, x):
        # print("shape of x: ", x.shape)
        out = F.relu(self.conv1(x))
        # print("shape of first out: ", out.shape)
        out = F.relu(self.conv2(out))
        # print("shape of second out: ", out.shape)
        res = self.res_conv(x)
        # print("shape of res: ", res.shape)
        return out + res

class WaveNet(nn.Module):
    def __init__(self, in_channels, out_channels, num_blocks, dilations):
        super(WaveNet, self).__init__()
        self.initial_conv = nn.Conv1d(in_channels, 32, kernel_size=2, padding=1)
        self.blocks = nn.ModuleList([WaveNetBlock(32, dilation) for dilation in dilations])
        self.final_conv1 = nn.Conv1d(32, 128, kernel_size=2, padding=0)
        self.final_conv2 = nn.Conv1d(128, 256, kernel_size=2, padding=0)
        self.fc1 = nn.Linear(256, 128)
        self.fc2 = nn.Linear(128, out_channels)
        
    def forward(self, x):
        x = F.relu(self.initial_conv(x))
        for block in self.blocks:
            # print("enter the block loop")
            x = block(x)
        x = F.relu(self.final_conv1(x))
        x = F.relu(self.final_conv2(x))
        x = x[:, :, -1]  # Get the last time step
        x = F.relu(self.fc1(x))
        x = self.fc2(x)
        return x

input_channels = 4  # Number of features
output_channels = 1  # Predicting a single value (glucose level)
num_blocks = 4  # Number of WaveNet blocks
dilations = [2**i for i in range(num_blocks)]  # Dilation rates: 1, 2, 4, 8

model = WaveNet(input_channels, output_channels, num_blocks, dilations)
print(model)

# Example of how to define the loss and optimizer
criterion = nn.MSELoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.0008)


WaveNet(
  (initial_conv): Conv1d(4, 32, kernel_size=(2,), stride=(1,), padding=(1,))
  (blocks): ModuleList(
    (0): WaveNetBlock(
      (conv1): Conv1d(32, 32, kernel_size=(2,), stride=(1,))
      (conv2): Conv1d(32, 32, kernel_size=(2,), stride=(1,), padding=(1,))
      (res_conv): Conv1d(32, 32, kernel_size=(1,), stride=(1,))
    )
    (1): WaveNetBlock(
      (conv1): Conv1d(32, 32, kernel_size=(2,), stride=(1,), dilation=(2,))
      (conv2): Conv1d(32, 32, kernel_size=(2,), stride=(1,), padding=(2,), dilation=(2,))
      (res_conv): Conv1d(32, 32, kernel_size=(1,), stride=(1,))
    )
    (2): WaveNetBlock(
      (conv1): Conv1d(32, 32, kernel_size=(2,), stride=(1,), dilation=(4,))
      (conv2): Conv1d(32, 32, kernel_size=(2,), stride=(1,), padding=(4,), dilation=(4,))
      (res_conv): Conv1d(32, 32, kernel_size=(1,), stride=(1,))
    )
    (3): WaveNetBlock(
      (conv1): Conv1d(32, 32, kernel_size=(2,), stride=(1,), dilation=(8,))
      (conv2): Conv1d(32, 32, kernel_size=(2

In [38]:
for inputs, targets in train_loader:
    print("Input tensor shape:", inputs.shape)
    print("Input tensor total elements:", inputs.numel())
    print("Target tensor shape:", targets.shape)
    print("Sequence length:", inputs.shape[1])
    break


Input tensor shape: torch.Size([32, 16, 4])
Input tensor total elements: 2048
Target tensor shape: torch.Size([32, 1])
Sequence length: 16


In [39]:

# Training Loop
num_epochs = 200
for epoch in range(num_epochs):
    model.train()
    for inputs, targets in train_loader:
        optimizer.zero_grad()
        outputs = model(inputs.permute(0, 2, 1))  # Permute to match (batch, channels, seq_len)
        loss = criterion(outputs, targets)
        loss.backward()
        optimizer.step()

    model.eval()
    val_loss = 0
    with torch.no_grad():
        for inputs, targets in val_loader:
            outputs = model(inputs.permute(0, 2, 1))  # Permute to match (batch, channels, seq_len)
            loss = criterion(outputs, targets)
            val_loss += loss.item()
    
    print(f'Epoch {epoch+1}, Validation Loss: {val_loss / len(val_loader)}')

/dartfs-hpc/rc/home/j/f007g3j/miniconda3/lib/python3.12/site-packages/torch/nn/modules/loss.py:608: UserWarning: Using a target size (torch.Size([32])) that is different to the input size (torch.Size([32, 1])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.mse_loss(input, target, reduction=self.reduction)
/dartfs-hpc/rc/home/j/f007g3j/miniconda3/lib/python3.12/site-packages/torch/nn/modules/loss.py:608: UserWarning: Using a target size (torch.Size([30])) that is different to the input size (torch.Size([30, 1])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.mse_loss(input, target, reduction=self.reduction)


Epoch 1, Validation Loss: 4863.485068321228


KeyboardInterrupt: 